### Probability Map  Generation
This file provides direct access to model outputs for downstream analysis.

In [1]:
from os.path import join
from gelgenie.segmentation.evaluation.core_functions import segment_and_plot, model_predict_and_process
from gelgenie.segmentation.helper_functions.general_functions import create_dir_if_empty
import toml
import torch
from gelgenie.segmentation.networks import model_configure
from gelgenie.segmentation.helper_functions.stat_functions import load_statistics
from torch.utils.data import DataLoader
from gelgenie.segmentation.data_handling.dataloaders import ImageDataset, ImageMaskDataset
from tqdm.notebook import tqdm
import numpy as np

def model_eval_load(exp_folder, eval_epoch):

    model_config = toml.load(join(exp_folder, 'config.toml'))['model']
    model, _, _ = model_configure(**model_config)
    if eval_epoch == 'best':
        stats = load_statistics(join(exp_folder, 'training_logs'), 'training_stats.csv', config='pd')
        sel_epoch = stats['Epoch'][stats['Dice Score'].idxmax()]
    else:
        sel_epoch = eval_epoch

    checkpoint = torch.load(f=join(exp_folder, 'checkpoints', 'checkpoint_epoch_%s.pth' % sel_epoch),
                            map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint['network'])
    model.eval()

    return model



In [3]:
model_and_epoch = [('unet_global_padding_nov_4', 'best'),
                   ('unet_dec_21', 'best'),
                   ('unet_dec_21_extended_set', '600')]
experiment_names, eval_epochs = zip(*model_and_epoch)

model_folder = '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/segmentation_models/December 2023'
output_folder = '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/probability_map_samples/v2'

visual_out = join(output_folder, 'visual_segmentation_results')

models = []
for experiment, eval_epoch in zip(experiment_names, eval_epochs):
    if 'nov_4' in experiment:
       exp_folder = join('/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/segmentation_models/November 2023', experiment)
    else:
       exp_folder = join(model_folder, experiment)
    model = model_eval_load(exp_folder, eval_epoch)
    models.append(model)


In [10]:
input_folder = ['/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels/test_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels_2/test_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/nathan_gels/test_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/quantitation_ladder_gels/test_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/lsdb_gels/test_images']
mask_folder = [
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels/test_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels_2/test_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/nathan_gels/test_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/quantitation_ladder_gels/test_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/lsdb_gels/test_masks']

dataset = ImageMaskDataset(input_folder, mask_folder, 1, padding=False, individual_padding=True)
dataloader = DataLoader(dataset, shuffle=False, batch_size=1, num_workers=0, pin_memory=True)
model_outputs = join(output_folder, 'direct_model_outputs_test_set')
create_dir_if_empty(model_outputs)

for im_index, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    np_mask = batch['mask'].detach().squeeze().cpu().numpy()
    name = batch['image_name'][0]
    all_model_outputs = []
    sel_image_folder = join(model_outputs, name)
    create_dir_if_empty(sel_image_folder)
    for model_name, model in zip(experiment_names, models):
        create_dir_if_empty(join(sel_image_folder, model_name))
        seg_mask, seg_ordered_mask = model_predict_and_process(model, batch['image'])
        np.save(join(sel_image_folder, model_name, 'seg_mask.npy'), seg_mask.detach().squeeze().cpu().numpy())
        np.save(join(sel_image_folder, model_name, 'seg_mask_one_hot.npy'), seg_ordered_mask)
        np.save(join(sel_image_folder, model_name, 'true_mask.npy'), np_mask)

In [11]:
input_folder = ['/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels/val_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels_2/val_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/nathan_gels/val_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/quantitation_ladder_gels/val_images',
                '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/lsdb_gels/val_images']
mask_folder = [
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels/val_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/matthew_gels_2/val_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/nathan_gels/val_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/quantitation_ladder_gels/val_masks',
    '/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/data/processed_gels/maximal_set/lsdb_gels/val_masks']

dataset = ImageMaskDataset(input_folder, mask_folder, 1, padding=False, individual_padding=True)
dataloader = DataLoader(dataset, shuffle=False, batch_size=1, num_workers=0, pin_memory=True)
model_outputs = join(output_folder, 'direct_model_outputs_val_set')
create_dir_if_empty(model_outputs)

for im_index, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    np_mask = batch['mask'].detach().squeeze().cpu().numpy()
    name = batch['image_name'][0]
    all_model_outputs = []
    sel_image_folder = join(model_outputs, name)
    create_dir_if_empty(sel_image_folder)
    for model_name, model in zip(experiment_names, models):
        create_dir_if_empty(join(sel_image_folder, model_name))
        seg_mask, seg_ordered_mask = model_predict_and_process(model, batch['image'])
        np.save(join(sel_image_folder, model_name, 'seg_mask.npy'), seg_mask.detach().squeeze().cpu().numpy())
        np.save(join(sel_image_folder, model_name, 'seg_mask_one_hot.npy'), seg_ordered_mask)
        np.save(join(sel_image_folder, model_name, 'true_mask.npy'), np_mask)

In [8]:
# testing direct model outputs
segment_and_plot(models, experiment_names, input_folder, visual_out)